In [ ]:
from langchain_groq import ChatGroq

# Step1: Web scrapping

use langchain WebBaseLoader

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

In [ ]:
from langchain_core.prompts import PromptTemplate

In [ ]:
prompt_extract = PromptTemplate.from_template(
    """
            ### SCRAPED TEXT FROM WEBSITE:
            {page_data}
            ### INSTRUCTION:
            The scraped text is from the career's page of a website.
            Your job is to extract the job postings and return them in JSON format containing the following keys: `role`, `experience`, `skills` and `description`.
            Only return the valid JSON.
            ### VALID JSON (NO PREAMBLE):
            """
)

In [ ]:
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={"page_data":page_data})
print(res.content)

In [ ]:
%pip install -qU langchain langchain-openai

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
print(json_res)
type(json_res)

Inssert the data in chromaDB

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("my_portfolio.csv")
df


Now iterate over the above data to store them in database(ChromaDB)

In [ ]:
pip install chromadb

In [ ]:
import chromadb as db
import uuid


In [ ]:
client = db.PersistentClient("VectoreStore")
## we used persistentClient what that does is create a folder and stores the data in that folder
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
  for _, row in df.iterrows():
    collection.add(documents = row["Techstack"],
                   metadatas={"links":row["Links"]},
                   ids=[str(uuid.uuid4())])

now will write a Query

In [ ]:
links = collection.query(
    query_texts=job["skills"],
    n_results=2,
).get("metadatas")
links

In [ ]:
job = json_res[0]
job["skills"]


Create a prompt template for generating emails.

In [ ]:
prompt_email = PromptTemplate.from_template(
    """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are Kalash, Founder and developer of CloudDrop. CloudDrop is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools.
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
        process optimization, cost reduction, and heightened overall efficiency.
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of CloudDrop
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Kalash, founder and developer of CloudDrop.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description":str(job), "link_list":links})
print(res.content)